In [1]:
from icepy4d.utils import initialization
from icepy4d.classes import Image
from icepy4d.matching import SuperGlueMatcher, LOFTRMatcher, Quality, TileSelection, GeometricVerification

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Even though this step is not mandatory, it is suggested to setup a logger to see the output of the matching process. If no logger is setup, the output of the process is suppressed.
The logger can be setup as follows:

In [2]:
initialization.setup_logger()

The first step is to load the images as numpy arrays.  
We will use the Image class implemented in ICEpy4D, which allows for creating an Image instance by passing the path to the image file as `Image('path_to_image')`.  
Creating the Image instance will read the exif data of the image and store them in the Image object. The actual image value is read when the Image.value proprierty is accessed.
Alternatevely, one can also use OpencCV imread function to read the image as a numpy array (pay attention to the channel order, that should be RGB, while Opencv uses BGR).

In [3]:
image0 = Image('../data/img/p1/IMG_2637.jpg').value
image1 = Image('../data/img/p2/IMG_1112.jpg').value

### SuperGlue matching

For running the matching with SuperGlue, a new SuperGlueMatcher object must be initialized with the parameters for SuperGlue matching (see the documentation of the class for more details). The parameters are given as a dictionary, and the default values are used if nothing is given.

In [4]:
matching_cfg = {
    "weights": "outdoor",
    "keypoint_threshold": 0.0001,
    "max_keypoints": 8192,
    "match_threshold": 0.2,
    "force_cpu": False,
}

matcher = SuperGlueMatcher(matching_cfg)
matcher.match(
    image0,
    image1,
    quality=Quality.HIGH,
    tile_selection=TileSelection.PRESELECTION,
    grid=[4, 3],
    overlap=200,
    do_viz_matches=True,
    do_viz_tiles=False,
    save_dir = "./matches/superglue_matches",
    geometric_verification=GeometricVerification.PYDEGENSAC,
    threshold=1.5,
)

2023-09-11 11:24:49 | [INFO    ] Running inference on device cuda
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
2023-09-11 11:24:50 | [INFO    ] Matching by tiles...
2023-09-11 11:24:50 | [INFO    ] Matching tiles by preselection tile selection
2023-09-11 11:24:50 | [INFO    ] Matching completed.
2023-09-11 11:24:51 | [INFO    ]  - Matching tile pair (3, 2)
2023-09-11 11:24:53 | [INFO    ]  - Matching tile pair (4, 7)
2023-09-11 11:24:55 | [INFO    ]  - Matching tile pair (5, 7)
2023-09-11 11:24:57 | [INFO    ]  - Matching tile pair (5, 8)
2023-09-11 11:25:00 | [INFO    ]  - Matching tile pair (6, 6)
2023-09-11 11:25:02 | [INFO    ]  - Matching tile pair (6, 9)
2023-09-11 11:25:05 | [INFO    ]  - Matching tile pair (7, 6)
2023-09-11 11:25:07 | [INFO    ]  - Matching tile pair (7, 7)
2023-09-11 11:25:09 | [INFO    ]  - Matching tile pair (7, 9)
2023-09-11 11:25:12 | [INFO    ]  - Matching tile pair (7, 10)
2023-09-11 11:25:14 | [INFO    ]  - Matching tile pair (8, 7

True

The matches with their descriptors and scores are saved in the matcher object.
All the results are saved as numpy arrays with float32 dtype.
They can be accessed as follows:

In [17]:
mktps1 = matcher.mkpts0
mktps1 = matcher.mkpts1

print(f"Number of matches: {len(mktps1)}")
print(f"Matchs on image0 (first 10):\n{mktps1[0:10]}")
print(f"Matchs on image1 (first 10):\n{mktps1[0:10]}")

Number of matches: 2408
Matchs on image0 (first 10):
[[5249.  244.]
 [5267.  204.]
 [5348.   84.]
 [5268.  224.]
 [5253.  280.]
 [5364.   98.]
 [5268.  236.]
 [5407.   43.]
 [5302.  179.]
 [5274.  258.]]
Matchs on image1 (first 10):
[[5249.  244.]
 [5267.  204.]
 [5348.   84.]
 [5268.  224.]
 [5253.  280.]
 [5364.   98.]
 [5268.  236.]
 [5407.   43.]
 [5302.  179.]
 [5274.  258.]]


### LOFTR matching

In [ ]:
matcher = LOFTRMatcher()
matcher.match(
    image0,
    image1,
    quality=Quality.HIGH,
    tile_selection=TileSelection.PRESELECTION,
    grid=[5, 4],
    overlap=100,
    save_dir= "./matches/LOFTR_matches",
    geometric_verification=GeometricVerification.PYDEGENSAC,
    threshold=1.5,
)

In [6]:
# Clean up result folders

import shutil

shutil.rmtree("./matches")
shutil.rmtree("./logs")

FileNotFoundError: [Errno 2] No such file or directory: './matches'